# LangSmith

In [ ]:
!pip install -qU langchain huggingface_hub langchain-community langchainhub langsmith langchain-ai21 langchain-cohere
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.llms import HuggingFaceEndpoint
from langchain_cohere.llms import Cohere
from langchain_cohere import ChatCohere
from langchain_core.tools import tool
from langchain import hub
from uuid import uuid4
import os

In [ ]:
os.environ["COHERE_API_KEY"] = "your_api_key"

In [ ]:
chat = ChatCohere()
llm = Cohere()

In [ ]:
unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"LangChain_Presentation"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "your_api_key"

In [ ]:
from langsmith import Client
client = Client()

In [ ]:
example_inputs = [
    ("What is the largest mammal?", "The blue whale"),
    ("What do mammals and birds have in common?", "They are both warm-blooded"),
    ("What are reptilees knwown for?", "Having scales"),
    ("What's the main characteristic of amphibians?", "They live both in water and on land"),
]

In [ ]:
example_inputs = [
    ("Which mammal is known for its ability to fly?", "The bat")
    ("What is the primary feature of fish?", "They have gills to breathe underwater")
    ("What do all insects have in common?", "They have three main body parts: head, thorax, and abdomen")
    ("What is a defining characteristic of birds?", "They have feathers")
]

In [ ]:
dataset_name = "Elementary Animal Questions"

In [ ]:
dataset = client.create_dataset(
    dataset_name="Elementary Animal Questions",
    description="Questions and answers about animal phylogenetics."
)

In [ ]:
for input_prompt, output_answer in example_inputs:
  client.create_example(
      inputs={"question": input_prompt},
      outputs={"answer": output_answer},
      dataset_id=dataset.id
  )

# Evaluation

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langsmith.evaluation import evaluate

# Target task definition
prompt = ChatPromptTemplate.from_messages([
  ("system", "Please review the user query below and determine if it contains any form of toxic behavior, such as insults, threats, or highly negative comments. Respond with 'Toxic' if it does, and 'Not toxic' if it doesn't."),
  ("user", "{question}")
])

output_parser = StrOutputParser()

chain = prompt | chat | output_parser

# The name or UUID of the LangSmith dataset to evaluate on.
# Alternatively, you can pass an iterator of examples
data = "Elementary Animal Questions"

# A string to prefix the experiment name with.
# If not provided, a random string will be generated.
experiment_prefix = "Elementary Animal Questions"

# Evaluate the target task
results = evaluate(
  chain.invoke,
  data=data,
  experiment_prefix=experiment_prefix,
)

View the evaluation results for experiment: 'Elementary Animal Questions-6ea03e3e' at:
https://smith.langchain.com/o/15793a46-90bc-5ff7-86c1-16f9efe55fab/datasets/c9afb3a0-9169-4169-b94f-9101893a9052/compare?selectedSessions=a9f1d12c-de50-4109-a42f-58774e1c4bf7




0it [00:00, ?it/s]

# Toolkits

In [ ]:
from langchain_community.agent_toolkits.nasa.toolkit import NasaToolkit
from langchain_community.utilities.nasa import NasaAPIWrapper

In [ ]:
nasa = NasaAPIWrapper()
toolkit = NasaToolkit.from_nasa_api_wrapper(nasa)

In [ ]:
for tool in toolkit.tools:
  print('Nome da tool: ', tool.name)
  print('Descrição da tool: ', tool.description)

Nome da tool:  Search NASA Image and Video Library media
Descrição da tool:  
    This tool is a wrapper around NASA's search API, useful when you need to search through NASA's Image and Video Library. 
    The input to this tool is a query specified by the user, and will be passed into NASA's `search` function.
    
    At least one parameter must be provided.

    There are optional parameters that can be passed by the user based on their query
    specifications. Each item in this list contains pound sign (#) separated values, the first value is the parameter name, 
    the second value is the datatype and the third value is the description: {{

        - q#string#Free text search terms to compare to all indexed metadata.
        - center#string#NASA center which published the media.
        - description#string#Terms to search for in “Description” fields.
        - description_508#string#Terms to search for in “508 Description” fields.
        - keywords #string#Terms to search for

In [ ]:
from langchain.agents import AgentExecutor, create_react_agent

prompt = hub.pull("hwchase17/react")
agent_nasa = create_react_agent(llm, toolkit.get_tools(), prompt)
agent_executor_nasa = AgentExecutor(agent=agent_nasa, tools=toolkit.get_tools(), verbose=True)

In [ ]:
agent_executor_nasa.invoke({"input": "Find the metadata for the image PIA13005 "})



> Entering new AgentExecutor chain...
 I need to use the Get NASA Image and Video Library media metadata manifest tool to find the metadata for image PIA13005. The tool requires a NASA ID, so I should search for that ID using the Search NASA Image and Video Library media tool. 
Action: Search NASA Image and Video Library media
Action Input: {"q": "PIA13005"}{"collection": {"version": "1.0", "href": "http://images-api.nasa.gov/search?q=PIA13005", "items": [{"href": "https://images-assets.nasa.gov/image/PIA13005/collection.json", "data": [{"center": "JPL", "title": "Orion Dreamy Stars", "nasa_id": "PIA13005", "date_created": "2010-04-01T21:00:00Z", "keywords": ["Spitzer Space Telescope"], "media_type": "image", "description_508": "A colony of hot, young stars is stirring up the cosmic scene in this new picture from NASA Spitzer Space Telescope. The image shows the Orion nebula, a happening place where stars are born.", "secondary_creator": "NASA/JPL-Caltech", "description": "A colony o

{'input': 'Find the metadata for the image PIA13005',
 'output': '{"collection": {"version": "1.0", "href": "http://images-api.nasa.gov/search?q=PIA13005", "items": [{"href": "https://images-assets.nasa.gov/image/PIA13005/collection.json", "data": [{"center": "JPL", "title": "Orion Dreamy Stars", "nasa_id": "PIA13005", "date_created": "2010-04-01T21:00:00Z", "keywords": ["Spitzer Space Telescope"], "media_type": "image", "description_508": "A colony of hot, young stars is stirring up the cosmic scene in this new picture from NASA Spitzer Space Telescope. The image shows the Orion nebula, a happening place where stars are born.", "secondary_creator": "NASA/JPL-Caltech", "description": "A colony of hot, young stars is stirring up the cosmic scene in this new picture from NASA Spitzer Space Telescope. The image shows the Orion nebula, a happening place where stars are born."}], "links": [{"href": "https://images-assets.nasa.gov/image/PIA13005/PIA13005~thumb.jpg", "rel": "preview", "render